# Activity 4.1 - Cleaning Walmart Data the OpenRefine Way

In this activity, you will practice what you learned in Lecture 4.5 by cleaning up a data set containing information on various Walmart locations.

In [1]:
import pandas as pd
from dfply import *

#### Initial Tasks

1. Try to read in the `./data/Walmart_United_States_&_Canada.csv` file and verify that you get an encoding error.  This means that the [character encoding](https://en.wikipedia.org/wiki/Character_encoding) isn't the default of `utf-8`.  The easiest way to fix this is to open and save the file in Visual Studio Code.

In [3]:
# Your code here
pd.read_csv('data/Walmart_United_States_&_Canada.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe8 in position 42095: invalid continuation byte

2. Read in the data to verify that the encoding is fixed, but that there are two more problems.  What are they?

In [7]:
#Your code here
(pd.read_csv('data/Walmart_United_States_&_Canada.csv')
>> head())

,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-1295"
0,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111"
1,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40..."
2,-114.138488,51.040871,"Walmart Supercentre; #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ..."
3,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,..."
4,-113.911590,51.040090,"Walmart Supercentre; #1136,","255 E Hills Blvd SE,Calgary ,AB T2A 4X7,(403) ..."


- No headers
- Multiple columns combined
    - Adress and phone number
    - Store type and store number


<font color="blue"> Your thoughts here </font>

3. Take another look at the file in VS Code and determine solutions to the two/three issues, then read in the data correctly by passing `pd.read_csv` the correct defaults for this data. **Note.** Leave the `"` in place for now, as they serve an important role here!

In [8]:
help(pd.read_csv) # This might help!

Help on function read_csv in module pandas.io.parsers.readers:

read_csv(filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]', sep=<no_default>, delimiter=None, header='infer', names=<no_default>, index_col=None, usecols=None, squeeze=None, prefix=<no_default>, mangle_dupe_cols=True, dtype: 'DtypeArg | None' = None, engine: 'CSVEngine | None' = None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=None, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression: 'CompressionOptions' = 'infer', thousands=None, decimal: 'str' = '.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, encoding_errors: 'str | None' = 'strict', dialect=None, error_bad_li

<font color="blue"> Your thoughts here </font>

**Add the header names in the read_csv option and seperate by comma.**

In [19]:
#Your code here
header = ['lat', 'long', 'store', 'address', 'province/state', 'phone']

walmart_locations = pd.read_csv('data/Walmart_United_States_&_Canada.csv',
                    names = header,
                    sep = ',')

walmart_locations.head()

,lat,long,store,address,province/state,phone
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-...",NaN,NaN
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",NaN,NaN
2,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40...",NaN,NaN
3,-114.138488,51.040871,"Walmart Supercentre; #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ...",NaN,NaN
4,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,...",NaN,NaN


## Cleaning up the store information.

As hinted at above, the presence of the `"` meant the two of the columns--one containing the store type/number and the other contain the address/phone number--are combined together.  This was done because some of these entries have a different number of variables.  For example, the store type/number column sometimes occasionally `Gas`.

In this part of the activity, you should apply the iterative OpenRefine approach to separate the information in the store column.

**Warning!** There is one entry that doesn't follow the same pattern as the rest.  You won't find this entry unless you carefully define/fix/eliminate patterns.

In [20]:
from more_dfply import case_when, ifelse
from more_dfply.facets import text_facet, text_filter

# Your code here.
(walmart_locations
>> select(X.store)
>> filter_by(text_filter(X.store, 'Gas', regex = True))
)



,store
3,"Walmart Supercentre; #3009,Gas,"
9,"Walmart; #3011,Gas,"
12,"Walmart Supercentre; #3010,Gas,"
39,"Walmart Supercentre; #3168,Gas,"
48,"Walmart Supercentre; #3151,Gas,"
...,...
6800,"Walmart Supercenter; #1653,Gas,"
6803,"Sam's Club; #6425,Gas,"
6804,"Sam's Club; #6430,Gas/Diesel,"
6806,"Walmart Supercenter; #4653,Gas,"


In [57]:
# View cell

(walmart_locations
>> select(X.store)
>> filter_by(~text_filter(X.store, 'Gas/Diesel'))
>> filter_by(~text_filter(X.store, 'Gas', regex = True))
>> filter_by(~text_filter(X.store, 'Walmart( Supercentre| Supercenter)?; #\d{4},', regex = True))
>> filter_by(~text_filter(X.store, 'Mkt', regex= True))
>> filter_by(~text_filter(X.store, '\sClub;', regex= True))
>> filter_by(~text_filter(X.store, 'Wm', regex = True))
>> filter_by(~text_filter(X.store, 'Murphy: USA', regex = True))
#>> filter_by(text_filter(X.store, ))
)

/Users/ah8941ed/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)


,store
893,"Walmart; Supercenter,#2507,"
2743,"Walmart Pickup;#9294,"
4400,"Walmart Supercenter;#5082,"
4830,"Walmart Supercenter;#5839,"


In [31]:
# Transform cell
(walmart_locations
>> select(X.store)
>> mutate(fuel = case_when((text_filter(X.store, 'Gas/Diesel'),
                            'Gas/Diesel'),
                            (text_filter(X.store, 'Gas'),
                            'Gas'),
                            (True, 'None')),
                store = X.store.replace('Gas(/Diesel)?,', '', regex = True))
)

,store,fuel
0,"Walmart Supercentre; #1050,",None
1,"Walmart Supercentre; #3658,",None
2,"Walmart Supercentre; #3013,",None
3,"Walmart Supercentre; #3009,",Gas
4,"Walmart; #1144,",None
...,...,...
6811,"Walmart Supercenter; #4471,",Gas
6812,"Walmart Supercenter; #1457,",None
6813,"Walmart Supercenter; #1461,",None
6814,"Walmart Supercenter; #1508,",None


## Preview of Coming Attractions

In this module's homework assignment, you will continue to clean up this data set.